In [1]:
import sys
import pandas as pd
import numpy as np
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
sns.set_context('notebook')
import warnings
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


/usr/local/lib/python3.7/site-packages/botocore/awsrequest.py:624: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class HeadersDict(collections.MutableMapping):
/usr/local/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [25]:
#### get all csv dataframes
US_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA


full_df = pd.concat([US_df, CA_df, DE_df, FR_df, GB_df, IN_df])




In [3]:
US_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [4]:
CA_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40881.000000,4.088100e+04,4.088100e+04,4.088100e+04,4.088100e+04
mean,20.795553,1.147036e+06,3.958269e+04,2.009195e+03,5.042975e+03
std,6.775054,3.390913e+06,1.326895e+05,1.900837e+04,2.157902e+04
min,1.000000,7.330000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.439020e+05,2.191000e+03,9.900000e+01,4.170000e+02
50%,24.000000,3.712040e+05,8.780000e+03,3.030000e+02,1.301000e+03
75%,24.000000,9.633020e+05,2.871700e+04,9.500000e+02,3.713000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


In [5]:
DE_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40840.000000,4.084000e+04,4.084000e+04,4.084000e+04,4.084000e+04
mean,20.705828,6.034553e+05,2.187550e+04,1.397136e+03,2.785857e+03
std,6.975813,2.348963e+06,1.018000e+05,1.457738e+04,1.745803e+04
min,1.000000,5.180000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,2.706875e+04,5.330000e+02,2.900000e+01,7.900000e+01
50%,24.000000,1.192770e+05,2.699000e+03,1.340000e+02,3.760000e+02
75%,24.000000,4.431015e+05,1.179625e+04,5.320000e+02,1.376000e+03
max,44.000000,1.138762e+08,4.924056e+06,1.470386e+06,1.084435e+06


In [6]:
FR_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40724.000000,4.072400e+04,4.072400e+04,4.072400e+04,4.072400e+04
mean,20.123809,4.199219e+05,1.738886e+04,8.149624e+02,1.832453e+03
std,6.984422,1.772130e+06,8.720509e+04,1.139219e+04,1.404321e+04
min,1.000000,2.230000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,1.697450e+04,3.380000e+02,1.800000e+01,5.600000e+01
50%,23.000000,7.372100e+04,1.892500e+03,8.300000e+01,2.350000e+02
75%,24.000000,2.708088e+05,7.969500e+03,3.350000e+02,8.410000e+02
max,44.000000,1.009116e+08,4.750254e+06,1.353661e+06,1.040912e+06


In [7]:
GB_df.describe()

,category_id,views,likes,dislikes,comment_count
count,38916.000000,3.891600e+04,3.891600e+04,3.891600e+04,3.891600e+04
mean,16.827937,5.911944e+06,1.345196e+05,7.612560e+03,1.308835e+04
std,7.752728,1.900121e+07,3.499893e+05,5.095683e+04,5.066740e+04
min,1.000000,8.510000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,10.000000,2.515272e+05,5.897000e+03,2.000000e+02,6.790000e+02
50%,20.000000,9.818890e+05,2.518250e+04,8.210000e+02,2.478000e+03
75%,24.000000,3.683628e+06,1.140892e+05,3.357500e+03,9.241500e+03
max,43.000000,4.245389e+08,5.613827e+06,1.944971e+06,1.626501e+06


In [8]:
IN_df.describe()

,category_id,views,likes,dislikes,comment_count
count,37352.000000,3.735200e+04,3.735200e+04,3.735200e+04,37352.00000
mean,21.576596,1.060478e+06,2.708272e+04,1.665082e+03,2676.99743
std,6.556593,3.184932e+06,9.714510e+04,1.607617e+04,14868.31713
min,1.000000,4.024000e+03,0.000000e+00,0.000000e+00,0.00000
25%,23.000000,1.239155e+05,8.640000e+02,1.080000e+02,81.00000
50%,24.000000,3.045860e+05,3.069000e+03,3.260000e+02,329.00000
75%,24.000000,7.992912e+05,1.377425e+04,1.019250e+03,1285.00000
max,43.000000,1.254322e+08,2.912710e+06,1.545017e+06,827755.00000


In [9]:
full_df.describe()

,category_id,views,likes,dislikes,comment_count
count,239662.000000,2.396620e+05,2.396620e+05,2.396620e+05,2.396620e+05
mean,20.002950,1.898465e+06,5.218776e+04,2.849046e+03,5.632043e+03
std,7.271423,8.746280e+06,1.950500e+05,2.703692e+04,2.951592e+04
min,1.000000,2.230000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,8.385125e+04,1.277000e+03,7.300000e+01,1.760000e+02
50%,24.000000,3.082685e+05,6.522000e+03,2.960000e+02,7.910000e+02
75%,24.000000,1.041631e+06,2.895575e+04,1.126000e+03,3.043000e+03
max,44.000000,4.245389e+08,5.613827e+06,1.944971e+06,1.626501e+06


In [10]:
###get most common tags
def get_most_common_tags(country_df):
    tags = country_df['tags'].to_string(index=False, header=False)
    split_tags = [i.replace('"', '') for i in tags.split("|")]
    stop_words = stopwords.words('english')
    filtered_tags = [word for word in split_tags if word not in stop_words]
    fdist = FreqDist(split_tags)
    most_popular_tags = fdist.most_common(1000)
    return fdist, dict(most_popular_tags)

fd1, us_most_common_tags = get_most_common_tags(US_df)
f2, ca_most_common_tags = get_most_common_tags(CA_df)
fd3, de_most_common_tags = get_most_common_tags(DE_df)
fd4, fr_most_common_tags = get_most_common_tags(FR_df)
fd5, gb_most_common_tags = get_most_common_tags(GB_df)
fd6, in_most_common_tags = get_most_common_tags(IN_df)



In [11]:
####do visualization for most popular tags
'''
word cloud 
bar graph 
'''

'\nword cloud \nbar graph \n'

In [12]:
## do sentiment analysis on each of the tags
positive_words_df = pd.read_fwf('positivewords.txt')
negative_words_df = pd.read_fwf('negativewords.txt')

def extract_features(words):
    return dict([(word, True) for word in words.split()])

def build_sentiment_analysis_model():
    positive_words = positive_words_df['positivewords'].values.tolist()
    negative_words = negative_words_df['negativewords'].values.tolist()
    pos_feats = [(extract_features(f), 'positive') for f in positive_words ]
    neg_feats = [(extract_features(f), 'negative') for f in negative_words ]
    dataset = pos_feats + neg_feats
    random.shuffle(dataset)
    cutoff = int(0.80 * len(dataset))
    train_data = dataset[:cutoff]
    test_data = dataset[cutoff:]

    classifier = NaiveBayesClassifier.train(train_data)
    # print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    return classifier

def execute_model(tags):
    classifications = {}
    classifier = build_sentiment_analysis_model()
    classifier.show_most_informative_features(5)
    for tag in tags:
        classified = classifier.classify(extract_features(tag))
        classifications[tag] = classified
    return classifications



In [13]:
classifications_us = execute_model(list(us_most_common_tags.keys()))
classifications_ca = execute_model(list(ca_most_common_tags.keys()))
classifications_de = execute_model(list(de_most_common_tags.keys()))
classifications_fr = execute_model(list(fr_most_common_tags.keys()))
classifications_gb = execute_model(list(gb_most_common_tags.keys()))
classifications_in = execute_model(list(in_most_common_tags.keys()))

Most Informative Features
             enviousness = True           positi : negati =      2.4 : 1.0
                 coolest = None           negati : positi =      1.0 : 1.0
               righteous = None           negati : positi =      1.0 : 1.0
                  snazzy = None           negati : positi =      1.0 : 1.0
                 happily = None           negati : positi =      1.0 : 1.0
Most Informative Features
                 coolest = None           negati : positi =      1.0 : 1.0
               righteous = None           negati : positi =      1.0 : 1.0
                  snazzy = None           negati : positi =      1.0 : 1.0
                 happily = None           negati : positi =      1.0 : 1.0
                  firmer = None           negati : positi =      1.0 : 1.0
Most Informative Features
             enviousness = True           positi : negati =      2.4 : 1.0
                 coolest = None           negati : positi =      1.0 : 1.0
               righteo

In [14]:
def get_sentiment_stats(classification, country):
    sentiments = list(classification.values())
    sentiments_df = pd.DataFrame(sentiments, columns=['Sentiment'])
    negatives = len(sentiments_df[sentiments_df['Sentiment'] =='negative'])
    positives = len(sentiments_df[sentiments_df['Sentiment'] =='positive'])
    total_len = len(sentiments_df)
    percentage_of_negative = negatives / total_len * 100
    percentage_of_positive = positives / total_len * 100
    print ("{}% of the tags found in the trending youtube videos for country {} were positive in nature, while {}% were negative in nature.\n".format(percentage_of_positive, country, percentage_of_negative))
    
get_sentiment_stats(classifications_us, "USA")
get_sentiment_stats(classifications_ca, "Canada")
get_sentiment_stats(classifications_de, "Denmakr")
get_sentiment_stats(classifications_fr, "France")
get_sentiment_stats(classifications_gb, "Great Britain")
get_sentiment_stats(classifications_in, "India")

2.5% of the tags found in the trending youtube videos for country USA were positive in nature, while 97.5% were negative in nature.

3.1% of the tags found in the trending youtube videos for country Canada were positive in nature, while 96.89999999999999% were negative in nature.

1.0999999999999999% of the tags found in the trending youtube videos for country Denmakr were positive in nature, while 98.9% were negative in nature.

1.3% of the tags found in the trending youtube videos for country France were positive in nature, while 98.7% were negative in nature.

1.3% of the tags found in the trending youtube videos for country Great Britain were positive in nature, while 98.7% were negative in nature.

1.9% of the tags found in the trending youtube videos for country India were positive in nature, while 98.1% were negative in nature.



In [15]:
## do visualization of sentiment analysis

In [30]:
def do_LDA_model(country_tags):
    tokenizer = RegexpTokenizer(r'\w+')

    # create English stop words list
    stop_words = stopwords.words('english')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # compile sample documents into a list
    tags = list(country_tags.keys())

    # list for tokenized documents in loop
    texts = []

    # loop through document list
    for i in tags:

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in stop_words]

        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]

        # add tokens to list
        texts.append(stemmed_tokens)

    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)

    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]

    # generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=100)
    print(ldamodel.print_topics(num_topics=3, num_words=10))

In [31]:
do_LDA_model(us_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(ca_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(de_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(fr_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(gb_most_common_tags)
print("-----------------------------------------------------")
do_LDA_model(in_most_common_tags)

### ----visualize maybe using pyLDAavis and maybe in a tabular format in a pandas df to show top topics

[(0, '0.011*"anim" + 0.008*"music" + 0.008*"youtub" + 0.007*"life" + 0.007*"origin" + 0.007*"trailer" + 0.006*"record" + 0.006*"puth" + 0.006*"disney" + 0.006*"smith"'), (1, '0.014*"movi" + 0.013*"new" + 0.011*"cat" + 0.007*"sport" + 0.007*"film" + 0.007*"scienc" + 0.007*"entertain" + 0.006*"jimmi" + 0.006*"super" + 0.006*"pop"'), (2, '0.019*"show" + 0.017*"video" + 0.016*"news" + 0.011*"buzzfe" + 0.010*"game" + 0.010*"makeup" + 0.008*"late" + 0.007*"beauti" + 0.006*"one" + 0.006*"last"')]
-----------------------------------------------------
[(0, '0.010*"video" + 0.010*"movi" + 0.009*"joe" + 0.009*"anim" + 0.009*"game" + 0.007*"nba" + 0.006*"top" + 0.006*"dr" + 0.006*"cnn" + 0.006*"channel"'), (1, '0.024*"news" + 0.018*"tv" + 0.014*"live" + 0.011*"buzzfe" + 0.007*"ab" + 0.007*"world" + 0.006*"cbn" + 0.006*"onlin" + 0.006*"latest" + 0.006*"jimmi"'), (2, '0.013*"show" + 0.010*"highlight" + 0.009*"late" + 0.008*"2018" + 0.008*"fox" + 0.008*"trump" + 0.007*"sport" + 0.007*"react" + 0.007*